In [1]:
import os
import re
import glob
import random
import numpy as np
import scipy
import scipy.io as sio
import scipy.ndimage as ndimage
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as patches
from matplotlib.lines import Line2D
from PIL import Image
from ipywidgets import interact, interactive, fixed, interact_manual
import braingeneers
import braingeneers.data.datasets_electrophysiology as ephys
from braingeneers.analysis.analysis import SpikeData, read_phy_files, randomize_raster
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter1d

In [2]:
import pickle
file = open('/home/jovyan/work/Human_Hippocampus/StitchedDataUpdated.pkl', 'rb')
stitched_data = pickle.load(file)
file.close()

In [4]:
sd = stitched_data

In [ ]:
def find_sttc_above_threshold(sd, threshold=0.3):
    sttc = np.zeros((sd.N, sd.N))
    for i in range(sd.N):
        for j in range(sd.N):
            sttc[i][j] = sd.spike_time_tiling(i, j)

    # Set diagonal values to -1
    np.fill_diagonal(sttc, -1)

    # Create a mask to filter STTC values above the given threshold
    mask = sttc > threshold

    # Find the indices of neuron pairs with STTC above the threshold
    neuron_pairs = np.transpose(np.where(mask))

    return neuron_pairs

In [ ]:
def find_suitable_pairs(sd, sttc_thresh=0.3, num_threshold):
    sttc = spike_time_tiling(sd)
    # Set diagonal values to -1
    np.fill_diagonal(sttc, -1)

    # Create a mask to filter STTC values above the given threshold
    mask = sttc > threshold

    # Find the indices of neuron pairs with STTC above the threshold
    neuron_pairs = np.transpose(np.where(mask))
    
    if neuron_pairs

In [ ]:
def plot_latency_histograms_for_random_pairs(sd, threshold=0.3, num_random_pairs=4):
    neuron_pairs = find_sttc_above_threshold(sd, threshold)

    # Get 'num_random_pairs' random indices to choose random neuron pairs
    random_indices = np.random.choice(neuron_pairs.shape[0], num_random_pairs, replace=False)
    random_neuron_pairs = neuron_pairs[random_indices]

    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    axs = axs.ravel()

    for i, neuron_pair in enumerate(random_neuron_pairs):
        neuron1, neuron2 = neuron_pair
        lates_n1 = sd.latencies_to_index(neuron1)
        lates_raw = lates_n1[neuron2]
        lates = list(filter(lambda x: x < 10 and x > -10, lates_raw))

        mean_latency = np.mean(lates)
        std_latency = np.std(lates)

        axs[i].hist(lates, bins=10, alpha=0.7, label='Latency')
        axs[i].axvline(mean_latency, color='red', linestyle='dashed', linewidth=2, label='Mean')
        axs[i].axvline(mean_latency - std_latency, color='green', linestyle='dashed', linewidth=2, label='Std -')
        axs[i].axvline(mean_latency + std_latency, color='green', linestyle='dashed', linewidth=2, label='Std +')

        axs[i].set_xlabel("Latency (ms)")
        axs[i].set_ylabel("Number of spike pairs")
        axs[i].set_title(f"Latency Histogram - Neuron Pair {neuron1} and {neuron2}")
        axs[i].legend()

    plt.tight_layout()
    plt.show()

# Call the function with your SpikeData object, the desired STTC threshold, and the number of random pairs to plot
plot_latency_histograms_for_random_pairs(sd, threshold=0.3, num_random_pairs=4)